In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.streaming import *
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType, LongType, DoubleType, FloatType, DataType, TimestampType

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
spark = (SparkSession
 .builder
 .appName("Python_CSV")
 .getOrCreate())

22/08/10 12:33:59 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.197.132 instead (on interface ens33)
22/08/10 12:33:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/10 12:34:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/10 12:34:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [13]:
df = spark.read.format('csv').option('inferSchema','true').option('header','true').load('/home/ubuntu/Spark/Spark_Work/python_file/data/csv_files/part-00000-c8f332a2-7f4a-4dc1-b55d-1de2a6b53d54-c000.csv')

In [14]:
df.schema

StructType(List(StructField(datetime,StringType,true),StructField(clicks,IntegerType,true),StructField(impressions,IntegerType,true),StructField(quantity,IntegerType,true),StructField(total_price,DoubleType,true)))

In [3]:

sc = StructType([
            StructField('datetime', TimestampType(), True),
            StructField('clicks', IntegerType(), True),
            StructField('impressions', IntegerType(), True),
            StructField('quantity', IntegerType(), True),
            StructField('total_price', DoubleType(), True)
         ])

print(sc)



StructType(List(StructField(datetime,TimestampType,true),StructField(clicks,IntegerType,true),StructField(impressions,IntegerType,true),StructField(quantity,IntegerType,true),StructField(total_price,DoubleType,true)))


In [4]:


csv_df = spark.readStream.option("maxFilesPerTrigger",5).schema(sc).format('csv').option('header','true').load('/home/ubuntu/Spark/Spark_Work/python_file/data/csv_files')


In [5]:

csv_df.printSchema()

csv_df.isStreaming


root
 |-- datetime: timestamp (nullable = true)
 |-- clicks: integer (nullable = true)
 |-- impressions: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- total_price: double (nullable = true)



True

In [6]:
windows_df = csv_df.withWatermark('datetime','2 minutes').groupBy(window('datetime','1 minutes')).agg(sum('clicks').alias("Clicks"),sum('impressions').alias("Impressions"),sum('quantity').alias("Quantity"),sum('total_price').alias("Total_price"))

df = windows_df

df = df.withColumn('Start',df.window.start)

df = df.withColumn('End',df.window.end)

df = df.drop('window')

#df.printSchema()

f_df = df.select('Clicks','Impressions','Quantity','Total_price','Start','End')



#f_df.printSchema()

#f_df.writeStream.outputMode("append").format("console").option('header','true').option("path", "/home/ubuntu/Spark/Spark_Work/python_file/data/parquet_files/").option("checkpointLocation", "/home/ubuntu/Spark/Spark_Work/python_file/data/checkpoint_parquet").start().awaitTermination()



In [8]:
def writer(f_df,batch):
    if(f_df.count() <= 0):
        print("Empty Data")
        print(f_df.count())
    else:
        print("Not Empty")
        f_df.show()
        f_df.coalesce(1).write.format('csv').mode('append').option("path", "/home/ubuntu/Spark/Spark_Work/python_file/data/stream_files/").option('header','true').save()

f_df.writeStream.outputMode('append').foreachBatch(writer).start()



22/08/10 12:36:58 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-179da31f-7e79-4e52-8937-17efc36a8ac6. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/08/10 12:36:58 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Empty Data


Empty Data


0


0


...

0


Empty Data


0


Empty Data


0


In [7]:


final_df = csv_df

final_df = final_df.write.outputMode("append").format("csv").option("path", "/home/ubuntu/Spark/Spark_Work/python_file/data/parquet_files/").option("checkpointLocation", "/home/ubuntu/Spark/Spark_Work/python_file/data/checkpoint_parquet").start().awaitTermination()



AnalysisException: 'write' can not be called on streaming Dataset/DataFrame

In [1]:

spark.stop()

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


NameError: name 'spark' is not defined

In [29]:
df = spark.read.format('parquet').option('inferSchema','true').option('header','true').load('/home/ubuntu/Spark/Spark_Work/python_file/data/parquet_files/part-00000-0a47e16d-699c-4d03-a996-548ade20aada-c000.snappy.parquet')

In [30]:
df.printSchema()

root
 |-- window: struct (nullable = true)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- Clicks: long (nullable = true)



In [31]:
df.show()

+------+------+
|window|Clicks|
+------+------+
+------+------+

